In [13]:
# Load Libraries
from pathlib import Path
import sys
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

In [25]:
# Set working directory
os.chdir(os.path.join(sys.path[0], ".."))

# Create processed data folder
data_folder = os.path.join(".", "data")
processed_folder = os.path.join(".", "data", "processed")

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

In [50]:
# Create folder for star data
star_folder = os.path.join(".", "data", "processed", "stars")

if not os.path.exists(star_folder):
    os.makedirs(star_folder)
    
# Read in star data
star_raw_path = os.path.join(data_folder, "raw", "star_classification.csv")
star_data = pd.read_csv(star_raw_path).query('`class` == "STAR"')

# Separate stars
star_data.to_csv(os.path.join(star_folder, "stars.csv"))

In [51]:
# Create folder for qso data
qso_folder = os.path.join(".", "data", "processed", "qso")

if not os.path.exists(qso_folder):
    os.makedirs(qso_folder)
    
# Read in qso data
qso_raw_path = os.path.join(data_folder, "raw", "star_classification.csv")
qso_data = pd.read_csv(qso_raw_path).query('`class` == "QSO"')

# Separate qso
qso_data.to_csv(os.path.join(qso_folder, "qso.csv"))

In [52]:
# Create folder for galaxy data
galaxy_folder = os.path.join(".", "data", "processed", "galaxy")

if not os.path.exists(galaxy_folder):
    os.makedirs(galaxy_folder)
    
# Read in galaxy data
galaxy_raw_path = os.path.join(data_folder, "raw", "star_classification.csv")
galaxy_data = pd.read_csv(galaxy_raw_path).query('`class` == "GALAXY"')

# Separate galaxy
galaxy_data.to_csv(os.path.join(galaxy_folder, "galaxy.csv"))

In [19]:
# Create folder for all data
total_folder = os.path.join(".", "data", "processed", "total")

if not os.path.exists(total_folder):
    os.makedirs(total_folder)
    
# Read in galaxy data
total_raw_path = os.path.join(data_folder, "raw", "star_classification.csv")
total_data = pd.read_csv(total_raw_path)
                         
# Separate into train test
X = total_data.drop('class', axis=1)
y = total_data['class']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

training_data = X_train

# Separate galaxy
galaxy_data.to_csv(os.path.join(total_folder, "train.csv"))

'/Users/dusty/Desktop/projects/astronomy/stellar_classification'

In [ ]:
# Separate QSO Data

In [ ]:
# Separate Galaxy Data